In [1]:
pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and pl

In [2]:
import pandas as pd
import numpy as np
from datasets import Dataset
import evaluate
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification
)
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {0: "negative", 1: "neutral", 2: "positive"}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_df = pd.read_csv("/content/drive/MyDrive/Newton/data/SemEval-2014/train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/Newton/data/SemEval-2014/val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Newton/data/SemEval-2014/test.csv")

In [5]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("fav-kky/FERNET-C5")

def tokenize_function(example):
    combined_input = f"aspect: {example['aspect']} text: {example['text']}"

    encoding = tokenizer(
        combined_input,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    # Convert label from string to int
    encoding["label"] = label2id[example["label"]]
    return encoding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/566 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/874k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
train_dataset = train_dataset.map(tokenize_function, batched=False)
val_dataset = val_dataset.map(tokenize_function, batched=False)
test_dataset = test_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/1619 [00:00<?, ? examples/s]

Map:   0%|          | 0/462 [00:00<?, ? examples/s]

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    "fav-kky/FERNET-C5",
    num_labels=3
)

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at fav-kky/FERNET-C5 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=200,
    warmup_ratio=0.06,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fialavi2 (fialavi2-czech-technical-university-in-prague) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.039521,0.458874
2,0.918000,0.907826,0.625541
3,0.918000,0.832659,0.673160
4,0.550400,1.228555,0.575758
5,0.550400,1.275064,0.619048
6,0.303400,1.218251,0.668831
7,0.303400,1.287901,0.692641
8,0.192300,1.486002,0.679654
9,0.192300,1.581194,0.688312
10,0.110500,1.690470,0.664502


TrainOutput(global_step=1020, training_loss=0.4078277226756601, metrics={'train_runtime': 933.5708, 'train_samples_per_second': 17.342, 'train_steps_per_second': 1.093, 'total_flos': 2129903116508160.0, 'train_loss': 0.4078277226756601, 'epoch': 10.0})

In [12]:
val_results = trainer.evaluate()
print(val_results)

{'eval_loss': 1.2879012823104858, 'eval_accuracy': 0.6926406926406926, 'eval_runtime': 6.8446, 'eval_samples_per_second': 67.498, 'eval_steps_per_second': 4.237, 'epoch': 10.0}


In [13]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 1.1234370470046997, 'eval_accuracy': 0.728448275862069, 'eval_runtime': 3.5186, 'eval_samples_per_second': 65.936, 'eval_steps_per_second': 4.263, 'epoch': 10.0}


In [14]:
trainer.save_model("/content/drive/MyDrive/Newton/models/FERNET-C5")

In [15]:
tokenizer.save_pretrained("/content/drive/MyDrive/Newton/models/FERNET-C5")

('/content/drive/MyDrive/Newton/models/FERNET-C5/tokenizer_config.json',
 '/content/drive/MyDrive/Newton/models/FERNET-C5/special_tokens_map.json',
 '/content/drive/MyDrive/Newton/models/FERNET-C5/vocab.txt',
 '/content/drive/MyDrive/Newton/models/FERNET-C5/added_tokens.json',
 '/content/drive/MyDrive/Newton/models/FERNET-C5/tokenizer.json')

In [24]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Newton/models/FERNET-C5")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Newton/models/FERNET-C5")

## Vyhodnocovací metriky

In [17]:
def compute_metrics_accuracy(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer_accuracy = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_accuracy
)

In [18]:
test_results_accuracy = trainer_accuracy.evaluate()
print(test_results_accuracy)

{'eval_loss': 1.1234371662139893, 'eval_model_preparation_time': 0.003, 'eval_accuracy': 0.728448275862069, 'eval_runtime': 4.8459, 'eval_samples_per_second': 47.876, 'eval_steps_per_second': 5.984}


In [19]:
def compute_metrics_weightedf1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

trainer_weightedf1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_weightedf1
)

In [23]:
test_results_weightedf1 = trainer_weightedf1.evaluate()
print(test_results_weightedf1)

{'eval_loss': 1.1234371662139893, 'eval_model_preparation_time': 0.0074, 'eval_f1': 0.7225038305597165, 'eval_runtime': 5.1967, 'eval_samples_per_second': 44.643, 'eval_steps_per_second': 5.58}


In [21]:
def compute_metrics_macrof1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

trainer_macrof1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_macrof1
)

In [22]:
test_results_macrof1 = trainer_macrof1.evaluate()
print(test_results_macrof1)

{'eval_loss': 1.1234371662139893, 'eval_model_preparation_time': 0.0028, 'eval_f1': 0.7040944014124091, 'eval_runtime': 4.9356, 'eval_samples_per_second': 47.006, 'eval_steps_per_second': 5.876}


## Praktické testy

In [25]:
def predict_sentiment(model, tokenizer, aspect, text):
    combined_input = f"aspect: {aspect} text: {text}"

    encoding = tokenizer(
        text=combined_input,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=256
    )
    with torch.no_grad():
        outputs = model(**encoding)

    logits = outputs.logits
    print(logits)
    predicted_class_id = logits.argmax(dim=1).item()
    return id2label[predicted_class_id]

### Sada L

In [26]:
example_aspect = "power"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L1: {prediction}, expected: positive")
example_aspect = "display"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L2: {prediction}, expected: negative")
example_aspect = "battery"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L3: {prediction}, expected: positive")

tensor([[-0.5684, -2.5401,  3.6906]])
Predicted sentiment for L1: positive, expected: positive
tensor([[ 0.3539, -2.5917,  2.8227]])
Predicted sentiment for L2: positive, expected: negative
tensor([[-0.6916, -2.5173,  3.7981]])
Predicted sentiment for L3: positive, expected: positive


### Sada R

In [27]:
example_aspect = "staff"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R1: {prediction}, expected: negative")
example_aspect = "table"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R2: {prediction}, expected: negative")
example_aspect = "food"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R3: {prediction}, expected: positive")
example_aspect = "beer"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R4: {prediction}, expected: positive")

tensor([[ 3.2337, -2.5925, -1.0153]])
Predicted sentiment for R1: negative, expected: negative
tensor([[ 3.3607, -2.4899, -1.2238]])
Predicted sentiment for R2: negative, expected: negative
tensor([[ 2.7648, -2.5934, -0.5081]])
Predicted sentiment for R3: negative, expected: positive
tensor([[ 3.1089, -2.4816, -1.0726]])
Predicted sentiment for R4: negative, expected: positive


### Sada M

In [28]:
example_aspect = "Osaka"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M1: {prediction}, expected: neutral")
example_aspect = "Saremi"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M2: {prediction}, expected: positive")
example_aspect = "Babtel"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M3: {prediction}, expected: negative")

tensor([[ 1.0927, -1.1400,  0.0799]])
Predicted sentiment for M1: negative, expected: neutral
tensor([[ 1.2098, -1.5683,  0.6120]])
Predicted sentiment for M2: negative, expected: positive
tensor([[ 1.1546, -1.1467,  0.1552]])
Predicted sentiment for M3: negative, expected: negative


### Sada O

In [29]:
example_aspect = "day"
example_text = "Today was a really good day, Anna asked me out for a date! Unfortunately the weather was awful, so we had to cancel it."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O1: {prediction}, expected: positive")
example_aspect = "weather"
example_text = "Today was a really good day, Anna asked me out for a date! Unfortunately the weather was awful, so we had to cancel it."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O2: {prediction}, expected: negative")

tensor([[-0.6040, -1.9180,  3.1546]])
Predicted sentiment for O1: positive, expected: positive
tensor([[-0.8539, -2.2190,  3.4706]])
Predicted sentiment for O2: positive, expected: negative
